In [29]:
!pip install langgraph langchain-groq langchain-core pydantic typing-extensions


In [30]:
import os
import re
from typing import Dict, Any, List, Literal
from dataclasses import dataclass
import json

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

# Try Groq, fallback to OpenAI if not available
try:
    from langchain_groq import ChatGroq
    LLM_PROVIDER = "groq"
except ImportError:
    try:
        from langchain_openai import ChatOpenAI
        LLM_PROVIDER = "openai"
    except ImportError:
        print("Neither Groq nor OpenAI available. Please install one of them.")

# Pydantic for type validation
from typing_extensions import TypedDict

print("✅ All imports successful!")

✅ All imports successful!


In [ ]:
class Config:
    """Configuration class for API keys and settings"""
    # Set your API key here or use environment variable
    GROQ_API_KEY = "YOUR API_KEY"  # Replace with your actual key
    OPENAI_API_KEY = "YOUR API_KEY"  # Alternative
    MODEL_NAME = "mixtral-8x7b-32768"  # Groq model
    OPENAI_MODEL = "gpt-3.5-turbo"     # OpenAI model

# Uncomment and set your actual API key
# os.environ["GROQ_API_KEY"] = "your-actual-groq-api-key"
# os.environ["OPENAI_API_KEY"] = "your-actual-openai-api-key"

print("✅ Configuration set!")

✅ Configuration set!


In [32]:
@tool
def plus(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract second number from first number."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide first number by second number."""
    if b == 0:
        raise ValueError("Division by zero is not allowed!")
    return a / b

print("✅ Mathematical tools defined!")
print(f"Available tools: {[plus.name, subtract.name, multiply.name, divide.name]}")


✅ Mathematical tools defined!
Available tools: ['plus', 'subtract', 'multiply', 'divide']


In [33]:
class AgentState(TypedDict):
    """State for the agent conversation"""
    messages: List[BaseMessage]
    next_action: str

print("✅ Agent state defined!")

✅ Agent state defined!


In [34]:
def setup_llm():
    """Initialize the LLM with proper configuration"""
    if LLM_PROVIDER == "groq":
        return ChatGroq(
            groq_api_key=Config.GROQ_API_KEY,
            model_name=Config.MODEL_NAME,
            temperature=0.1,
            max_tokens=1000
        )
    else:  # OpenAI fallback
        return ChatOpenAI(
            openai_api_key=Config.OPENAI_API_KEY,
            model_name=Config.OPENAI_MODEL,
            temperature=0.1,
            max_tokens=1000
        )

print(f"✅ LLM setup function ready! Using: {LLM_PROVIDER}")


✅ LLM setup function ready! Using: groq


In [35]:
def is_mathematical_query(query: str) -> bool:
    """Detect if a query is mathematical by looking for math keywords and patterns."""
    math_keywords = [
        'add', 'plus', 'sum', 'addition', '+',
        'subtract', 'minus', 'difference', 'subtraction', '-',
        'multiply', 'times', 'product', 'multiplication', '*', 'x',
        'divide', 'division', 'quotient', '/', '÷',
        'calculate', 'compute', 'what is', 'how much'
    ]

    query_lower = query.lower()

    # Check for math keywords
    has_math_keywords = any(keyword in query_lower for keyword in math_keywords)

    # Check for number patterns
    has_numbers = bool(re.search(r'\d+\.?\d*', query))

    # Check for mathematical expressions
    math_pattern = r'\d+\.?\d*\s*[+\-*/×÷]\s*\d+\.?\d*'
    has_math_expression = bool(re.search(math_pattern, query))

    return has_math_keywords and has_numbers or has_math_expression

def extract_math_operation(query: str) -> Dict[str, Any]:
    """Extract mathematical operation and numbers from query."""
    query_lower = query.lower()

    # Extract numbers from the query
    numbers = re.findall(r'\d+\.?\d*', query)
    numbers = [float(num) for num in numbers]

    if len(numbers) < 2:
        return {"operation": None, "operands": numbers}

    # Determine operation type - FIXED THE LOGIC HERE
    if any(word in query_lower for word in ['add', 'plus', 'sum']) or '+' in query:
        return {"operation": "plus", "operands": numbers[:2]}
    elif any(word in query_lower for word in ['subtract', 'minus']) or '-' in query:
        return {"operation": "subtract", "operands": numbers[:2]}
    elif any(word in query_lower for word in ['multiply', 'times', 'product']) or any(op in query for op in ['*', 'x']):
        return {"operation": "multiply", "operands": numbers[:2]}
    elif any(word in query_lower for word in ['divide', 'division', 'divided']) or any(op in query for op in ['/', '÷']):
        return {"operation": "divide", "operands": numbers[:2]}

    return {"operation": None, "operands": numbers}

print("✅ Query detection functions ready!")

# Test the detection
test_queries = ["What is 5 plus 3?", "Hello how are you?", "10 times 20"]
for q in test_queries:
    print(f"'{q}' -> Math query: {is_mathematical_query(q)}")


✅ Query detection functions ready!
'What is 5 plus 3?' -> Math query: True
'Hello how are you?' -> Math query: False
'10 times 20' -> Math query: True


In [36]:
def router_node(state: AgentState) -> AgentState:
    """Router node that decides where to send the query."""
    last_message = state["messages"][-1]

    if is_mathematical_query(last_message.content):
        return {
            "messages": state["messages"],
            "next_action": "tools"
        }
    else:
        return {
            "messages": state["messages"],
            "next_action": "chatbot"
        }

def chatbot_node(state: AgentState) -> AgentState:
    """Node for handling general queries using LLM."""
    last_message = state["messages"][-1]

    try:
        llm = setup_llm()
        system_prompt = """You are a helpful AI assistant. Answer the user's question clearly and concisely.
        If the user asks a mathematical question, let them know that you have specialized math functions available."""

        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{input}")
        ])

        chain = prompt | llm
        response = chain.invoke({"input": last_message.content})
        ai_response = response.content

    except Exception as e:
        # Fallback response if LLM fails
        ai_response = f"I'm a helpful AI assistant! You asked: '{last_message.content}'. " \
                     f"I can help with general questions and mathematical calculations. " \
                     f"(Note: LLM error - {str(e)[:50]}...)"

    updated_messages = state["messages"] + [AIMessage(content=ai_response)]

    return {
        "messages": updated_messages,
        "next_action": "end"
    }

def math_tools_node(state: AgentState) -> AgentState:
    """Node for handling mathematical operations using custom tools."""
    last_message = state["messages"][-1]
    query = last_message.content

    # Extract mathematical operation
    math_info = extract_math_operation(query)
    operation = math_info["operation"]
    operands = math_info["operands"]

    try:
        if operation and len(operands) >= 2:
            a, b = operands[0], operands[1]

            # Execute the appropriate mathematical function
            if operation == "plus":
                result = plus.invoke({"a": a, "b": b})
                response = f"The sum of {a} and {b} is {result}."
            elif operation == "subtract":
                result = subtract.invoke({"a": a, "b": b})
                response = f"The difference between {a} and {b} is {result}."
            elif operation == "multiply":
                result = multiply.invoke({"a": a, "b": b})
                response = f"The product of {a} and {b} is {result}."
            elif operation == "divide":
                try:
                    result = divide.invoke({"a": a, "b": b})
                    response = f"The quotient of {a} divided by {b} is {result}."
                except ValueError as e:
                    response = f"Error: {str(e)}"
            else:
                response = "I couldn't determine the mathematical operation. Please try rephrasing."
        else:
            response = "I couldn't extract the numbers or operation. Please specify two numbers and an operation."

    except Exception as e:
        response = f"I encountered an error while performing the calculation: {str(e)}"

    updated_messages = state["messages"] + [AIMessage(content=response)]

    return {
        "messages": updated_messages,
        "next_action": "end"
    }

print("✅ Graph node functions defined!")


✅ Graph node functions defined!


In [37]:
def route_after_router(state: AgentState) -> Literal["chatbot", "tools"]:
    """Route based on the next_action determined by router."""
    return state["next_action"]

print("✅ Routing logic defined!")

✅ Routing logic defined!


In [38]:
def create_agent_graph():
    """Create and configure the LangGraph state graph."""
    # Initialize the state graph
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("router", router_node)
    workflow.add_node("chatbot", chatbot_node)
    workflow.add_node("tools", math_tools_node)

    # Set entry point to router
    workflow.set_entry_point("router")

    # Add conditional edges from router
    workflow.add_conditional_edges(
        "router",
        route_after_router,
        {
            "chatbot": "chatbot",
            "tools": "tools"
        }
    )

    # Add edges to END
    workflow.add_edge("chatbot", END)
    workflow.add_edge("tools", END)

    # Compile the graph
    return workflow.compile()

print("✅ Graph construction function ready!")


✅ Graph construction function ready!


In [39]:
agent_graph = create_agent_graph()
print("✅ Agent graph created successfully!")

# Test with a simple query
def test_agent_query(query: str):
    """Test the agent with a single query"""
    print(f"\n🔄 Testing query: '{query}'")

    initial_state = {
        "messages": [HumanMessage(content=query)],
        "next_action": ""
    }

    try:
        final_state = agent_graph.invoke(initial_state)
        response = final_state["messages"][-1].content
        print(f"🤖 Response: {response}")
        return response
    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        print(error_msg)
        return error_msg


✅ Agent graph created successfully!


In [40]:
print("=" * 60)
print("🧮 TESTING MATHEMATICAL OPERATIONS")
print("=" * 60)

math_tests = [
    "What is 5 plus 3?",
    "10 minus 4",
    "How much is 6 times 7?",
    "What is 20 divided by 4?",
    "Calculate 15 + 25",
    "8 * 9",
    "100 / 5"
]

for query in math_tests:
    test_agent_query(query)


🧮 TESTING MATHEMATICAL OPERATIONS

🔄 Testing query: 'What is 5 plus 3?'
🤖 Response: The sum of 5.0 and 3.0 is 8.0.

🔄 Testing query: '10 minus 4'
🤖 Response: The difference between 10.0 and 4.0 is 6.0.

🔄 Testing query: 'How much is 6 times 7?'
🤖 Response: The product of 6.0 and 7.0 is 42.0.

🔄 Testing query: 'What is 20 divided by 4?'
🤖 Response: The quotient of 20.0 divided by 4.0 is 5.0.

🔄 Testing query: 'Calculate 15 + 25'
🤖 Response: The sum of 15.0 and 25.0 is 40.0.

🔄 Testing query: '8 * 9'
🤖 Response: The product of 8.0 and 9.0 is 72.0.

🔄 Testing query: '100 / 5'
🤖 Response: The quotient of 100.0 divided by 5.0 is 20.0.


In [41]:
print("\n" + "=" * 60)
print("💬 TESTING GENERAL CONVERSATIONS")
print("=" * 60)

general_tests = [
    "Hello, how are you?",
    "What is Python?",
    "Tell me about AI",
    "What's the weather like?",
    "Can you help me?"
]

for query in general_tests:
    test_agent_query(query)


💬 TESTING GENERAL CONVERSATIONS

🔄 Testing query: 'Hello, how are you?'
🤖 Response: I'm a helpful AI assistant! You asked: 'Hello, how are you?'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)

🔄 Testing query: 'What is Python?'
🤖 Response: I'm a helpful AI assistant! You asked: 'What is Python?'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)

🔄 Testing query: 'Tell me about AI'
🤖 Response: I'm a helpful AI assistant! You asked: 'Tell me about AI'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)

🔄 Testing query: 'What's the weather like?'
🤖 Response: I'm a helpful AI assistant! You asked: 'What's the weather like?'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 40

In [42]:
print("\n" + "=" * 60)
print("⚠️  TESTING ERROR HANDLING")
print("=" * 60)

error_tests = [
    "Divide 10 by 0",
    "What is 5 plus",  # Missing operand
    "Calculate something",  # Vague request
]

for query in error_tests:
    test_agent_query(query)


⚠️  TESTING ERROR HANDLING

🔄 Testing query: 'Divide 10 by 0'
🤖 Response: Error: Division by zero is not allowed!

🔄 Testing query: 'What is 5 plus'
🤖 Response: I couldn't extract the numbers or operation. Please specify two numbers and an operation.

🔄 Testing query: 'Calculate something'
🤖 Response: I'm a helpful AI assistant! You asked: 'Calculate something'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)


In [43]:
class MathematicalAgent:
    """Main agent class for interactive use"""

    def __init__(self):
        self.graph = create_agent_graph()
        self.conversation_history = []

    def chat(self, user_input: str) -> str:
        """Process user input and return response"""
        initial_state = {
            "messages": [HumanMessage(content=user_input)],
            "next_action": ""
        }

        try:
            final_state = self.graph.invoke(initial_state)
            response = final_state["messages"][-1].content

            # Update conversation history
            self.conversation_history.extend(final_state["messages"])

            return response
        except Exception as e:
            return f"❌ Error processing your request: {str(e)}"

    def reset_conversation(self):
        """Reset conversation history"""
        self.conversation_history = []

# Create agent instance
agent = MathematicalAgent()
print("✅ Interactive agent ready!")

✅ Interactive agent ready!


In [45]:
def manual_single_query():
    """Function to manually input a single query"""
    print("\n" + "="*50)
    print("🎯 MANUAL SINGLE QUERY MODE")
    print("="*50)

    user_input = input("👤 Enter your question: ").strip()

    if user_input:
        print(f"\n🔄 Processing: '{user_input}'")
        response = agent.chat(user_input)
        print(f"🤖 Response: {response}")
    else:
        print("❌ No input provided.")

def manual_batch_queries():
    """Function to manually input multiple queries at once"""
    print("\n" + "="*50)
    print("📝 MANUAL BATCH QUERY MODE")
    print("="*50)
    print("Enter multiple queries (one per line).")
    print("Press Enter twice when finished.")
    print("-" * 50)

    queries = []
    while True:
        line = input("Query: ").strip()
        if not line:
            break
        queries.append(line)

    if queries:
        print(f"\n🔄 Processing {len(queries)} queries...")
        print("-" * 50)

        for i, query in enumerate(queries, 1):
            print(f"\n{i}. Query: {query}")
            response = agent.chat(query)
            print(f"   Response: {response}")
    else:
        print("❌ No queries provided.")

def manual_interactive_chat():
    """Enhanced interactive chat with better controls"""
    print("\n" + "="*60)
    print("🤖 INTERACTIVE MATHEMATICAL AGENT CHAT")
    print("="*60)
    print("Commands:")
    print("  • Type your question or math problem")
    print("  • 'help' - Show available math operations")
    print("  • 'examples' - Show example queries")
    print("  • 'clear' - Clear conversation history")
    print("  • 'quit' or 'exit' - Exit chat")
    print("="*60)

    while True:
        try:
            user_input = input("\n👤 You: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
                print("👋 Thanks for chatting! Goodbye!")
                break

            elif user_input.lower() == 'help':
                print("🔧 Available Math Operations:")
                print("  • Addition: 'What is 5 plus 3?' or '5 + 3'")
                print("  • Subtraction: '10 minus 4' or '10 - 4'")
                print("  • Multiplication: '6 times 7' or '6 * 7'")
                print("  • Division: '20 divided by 4' or '20 / 4'")
                continue

            elif user_input.lower() == 'examples':
                print("📚 Example Queries:")
                print("Math:")
                print("  • 'What is 15 plus 25?'")
                print("  • 'Calculate 100 divided by 5'")
                print("  • '7 times 8'")
                print("General:")
                print("  • 'Hello, how are you?'")
                print("  • 'What is Python?'")
                print("  • 'Tell me a joke'")
                continue

            elif user_input.lower() == 'clear':
                agent.reset_conversation()
                print("🧹 Conversation history cleared!")
                continue

            elif not user_input:
                print("❓ Please enter a message or command.")
                continue

            # Process the query
            print("🔄 Processing...")
            response = agent.chat(user_input)
            print(f"🤖 Agent: {response}")

        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

In [46]:
def quick_test():
    """Run a quick test of all functionality"""
    print("🚀 QUICK FUNCTIONALITY TEST")
    print("=" * 40)

    test_cases = [
        ("Math: Addition", "What is 15 plus 25?"),
        ("Math: Subtraction", "20 minus 8"),
        ("Math: Multiplication", "7 times 6"),
        ("Math: Division", "100 divided by 5"),
        ("General: Greeting", "Hello there!"),
        ("General: Question", "What is Python?"),
        ("Error: Division by zero", "10 divided by 0")
    ]

    for test_name, query in test_cases:
        print(f"\n{test_name}:")
        print(f"  Query: {query}")
        response = agent.chat(query)
        print(f"  Response: {response}")

    print("\n✅ Quick test completed!")

# Run quick test automatically
quick_test()

🚀 QUICK FUNCTIONALITY TEST

Math: Addition:
  Query: What is 15 plus 25?
  Response: The sum of 15.0 and 25.0 is 40.0.

Math: Subtraction:
  Query: 20 minus 8
  Response: The difference between 20.0 and 8.0 is 12.0.

Math: Multiplication:
  Query: 7 times 6
  Response: The product of 7.0 and 6.0 is 42.0.

Math: Division:
  Query: 100 divided by 5
  Response: The quotient of 100.0 divided by 5.0 is 20.0.

General: Greeting:
  Query: Hello there!
  Response: I'm a helpful AI assistant! You asked: 'Hello there!'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)

General: Question:
  Query: What is Python?
  Response: I'm a helpful AI assistant! You asked: 'What is Python?'. I can help with general questions and mathematical calculations. (Note: LLM error - Error code: 400 - {'error': {'message': 'The model...)

Error: Division by zero:
  Query: 10 divided by 0
  Response: Error: Division by zero is n

In [47]:
def show_main_menu():
    """Display main menu for manual input options"""
    print("\n" + "="*60)
    print("🎛️  MANUAL INPUT MENU")
    print("="*60)
    print("Choose an option:")
    print("1. Single Query - Enter one question")
    print("2. Batch Queries - Enter multiple questions")
    print("3. Interactive Chat - Continuous conversation")
    print("4. Quick Test - Run automated tests")
    print("5. Exit")
    print("="*60)

    while True:
        try:
            choice = input("\n🎯 Enter your choice (1-5): ").strip()

            if choice == '1':
                manual_single_query()
            elif choice == '2':
                manual_batch_queries()
            elif choice == '3':
                manual_interactive_chat()
            elif choice == '4':
                quick_test()
            elif choice == '5':
                print("👋 Goodbye!")
                break
            else:
                print("❌ Invalid choice. Please enter 1-5.")

        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break

print("\n" + "=" * 60)
print("🎉 SETUP COMPLETE!")
print("=" * 60)
print("Your Fixed LangGraph Mathematical Agent is ready!")
print("\n📋 Available Manual Input Options:")
print("• manual_single_query() - Enter one question")
print("• manual_batch_queries() - Enter multiple questions")
print("• manual_interactive_chat() - Start interactive chat")
print("• show_main_menu() - Show all options")
print("• agent.chat('your message') - Direct single query")
print("\n🚀 To start with the menu: show_main_menu()")
print("🚀 To start chatting directly: manual_interactive_chat()")


🎉 SETUP COMPLETE!
Your Fixed LangGraph Mathematical Agent is ready!

📋 Available Manual Input Options:
• manual_single_query() - Enter one question
• manual_batch_queries() - Enter multiple questions
• manual_interactive_chat() - Start interactive chat
• show_main_menu() - Show all options
• agent.chat('your message') - Direct single query

🚀 To start with the menu: show_main_menu()
🚀 To start chatting directly: manual_interactive_chat()


In [48]:
manual_interactive_chat()


🤖 INTERACTIVE MATHEMATICAL AGENT CHAT
Commands:
  • Type your question or math problem
  • 'help' - Show available math operations
  • 'examples' - Show example queries
  • 'clear' - Clear conversation history
  • 'quit' or 'exit' - Exit chat

👤 You: 20 minus 8
🔄 Processing...
🤖 Agent: The difference between 20.0 and 8.0 is 12.0.

👤 You: 56 plus 56
🔄 Processing...
🤖 Agent: The sum of 56.0 and 56.0 is 112.0.

👤 You: exit
👋 Thanks for chatting! Goodbye!


In [49]:
show_main_menu()


🎛️  MANUAL INPUT MENU
Choose an option:
1. Single Query - Enter one question
2. Batch Queries - Enter multiple questions
3. Interactive Chat - Continuous conversation
4. Quick Test - Run automated tests
5. Exit

🎯 Enter your choice (1-5): 1

🎯 MANUAL SINGLE QUERY MODE
👤 Enter your question: 1200 divided by 10

🔄 Processing: '1200 divided by 10'
🤖 Response: The quotient of 1200.0 divided by 10.0 is 120.0.

🎯 Enter your choice (1-5): 5
👋 Goodbye!
